#Final Code :

In [ ]:
#!pip uninstall googletrans
!pip install googletrans==3.1.0a0
import googletrans
import re
import nltk
from nltk.tag import tnt
from nltk.corpus import indian
from googletrans import Translator

# Download the Hindi POS tagger
nltk.download('indian')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 KB 8.1 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16368 sha256=72e02f59197f6e8dffbe715e57ebce5b473997f28c754940a8eb0904c253b1c8
  Stored in directory: /root/.cache/pip/wheels/dd/59/af/8d6c96a719763990f1c548e36b17d9efdfb767f42f7ff39f53
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 4.0.0
 

[nltk_data] Downloading package indian to /root/nltk_data...
[nltk_data]   Unzipping corpora/indian.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
translator = Translator()
sentence_id = 0

text = "गर्मी के एक दिन में, जंगल के एक शेर को बहुत जोरों से भूख लगी. इसलिए वो इधर उधर खाने की तलाश करने लगा. कुछ देर खोजने के बाद उसे एक खरगोश मिला, लेकिन उसे खाने के बदले में उसे उसने छोड़ दिया क्यूंकि उसे वो बहुत ही छोटा लगा."
model_path = "/content/hindi.pos" #Copy hindi.pos from NLTK corpus

def train_hindi_model(model_path):
    train_data = indian.tagged_sents(model_path)
    tnt_pos_tagger = tnt.TnT()
    tnt_pos_tagger.train(train_data)
    return tnt_pos_tagger


def get_sentId(model_path):
    ids = re.compile('<Sentence\sid=\d+>')
    with open(model_path, "r+") as temp_f:
        content = temp_f.readlines()
        for i in content:
            id_found = (ids.findall(i))
            if id_found:
                id_found = str(id_found).replace("['<Sentence id=", "").replace(">']", "")
                id = int(id_found)
    id = id + 1
    return id


def tag_words(model,text):
    tagged = (model.tag(nltk.word_tokenize(text)))
    return tagged


def handle_UNK(tagged_words, model_path, sentence_id):
    with open(model_path, "r+") as f1:
        result_list = []
        for nep_word, tag in tagged_words:
            if tag == "Unk":
                x = translator.translate(nep_word)
                if x is not None:
                    str1 = str(x)
                    new_str = str1.split()
                    for j in new_str:
                        if re.search('^text=', j, re.I):
                            word = j.replace("text=", ",").replace(",", "")
                            word = str(word)
                            # pos=nltk.pos_tag(word)
                            pos = nltk.tag.pos_tag([word])
                            # print (i, pos)
                            for en_word, tag in pos:
                                result = nep_word + "_" + (tag) + " "
                                result_list.append(result)

            else:
                result = nep_word + "_" + (tag) + " "
                result_list.append(result)

        writing_word = str("\n<Sentence id=") + str(sentence_id) + ">\n"
        output = writing_word + "".join(result_list) + "\n</Sentence>\n</Corpora>"
        for line in f1.readlines():
            f1.write(line.replace("</Corpora>", ""))
        f1.write(output)


sentence_id = (get_sentId(model_path))
print (sentence_id)

model = train_hindi_model(model_path)
tagged_words = tag_words(model,text)

print ("=================================Tagged words=================================\n",tagged_words,"\n")

handle_UNK(tagged_words,model_path,sentence_id)

#retrain the model
model = train_hindi_model(model_path)
new_tagged_words =  tag_words(model,text)
print ("=================================New Tagged words=================================\n",new_tagged_words,"\n")

542
=================================Tagged words=================================
 [('गर्मी', 'Unk'), ('के', 'PREP'), ('एक', 'QFNUM'), ('दिन', 'NN'), ('में', 'PREP'), (',', 'PUNC'), ('जंगल', 'Unk'), ('के', 'PREP'), ('एक', 'QFNUM'), ('शेर', 'Unk'), ('को', 'PREP'), ('बहुत', 'INTF'), ('जोरों', 'NN'), ('से', 'PREP'), ('भूख', 'Unk'), ('लगी', 'VFM'), ('.', 'SYM'), ('इसलिए', 'RB'), ('वो', 'PRP'), ('इधर', 'Unk'), ('उधर', 'RB'), ('खाने', 'Unk'), ('की', 'PREP'), ('तलाश', 'NN'), ('करने', 'VNN'), ('लगा', 'VFM'), ('.', 'SYM'), ('कुछ', 'QF'), ('देर', 'Unk'), ('खोजने', 'Unk'), ('के', 'PREP'), ('बाद', 'PREP'), ('उसे', 'PRP'), ('एक', 'QFNUM'), ('खरगोश', 'Unk'), ('मिला', 'VFM'), (',', 'PUNC'), ('लेकिन', 'CC'), ('उसे', 'PRP'), ('खाने', 'Unk'), ('के', 'PREP'), ('बदले', 'NN'), ('में', 'PREP'), ('उसे', 'PRP'), ('उसने', 'PRP'), ('छोड़', 'Unk'), ('दिया', 'VAUX'), ('क्यूंकि', 'Unk'), ('उसे', 'PRP'), ('वो', 'PRP'), ('बहुत', 'INTF'), ('ही', 'RP'), ('छोटा', 'Unk'), ('लगा', 'VFM'), ('.', 'SYM')] 

===============

#Simple Model (Previous Model for Comparison) :

In [ ]:
import nltk
from nltk.corpus import indian
from nltk.tag import tnt

# Download the Hindi POS tagger
nltk.download('indian')
nltk.download('punkt')

[nltk_data] Downloading package indian to /root/nltk_data...
[nltk_data]   Package indian is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Train the POS tagger
data = indian.tagged_sents('hindi.pos')
tnt_pos_tagger = tnt.TnT()
tnt_pos_tagger.train(data)

# Define a function to tag Hindi text
def tag_hindi_text(text):
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    # Tag the tokens
    tagged_tokens = tnt_pos_tagger.tag(tokens)
    return tagged_tokens

# Test the function
text = "गर्मी के एक दिन में, जंगल के एक शेर को बहुत जोरों से भूख लगी. इसलिए वो इधर उधर खाने की तलाश करने लगा. कुछ देर खोजने के बाद उसे एक खरगोश मिला, लेकिन उसे खाने के बदले में उसे उसने छोड़ दिया क्यूंकि उसे वो बहुत ही छोटा लगा."
tagged_text = tag_hindi_text(text)
print(tagged_text)

[('गर्मी', 'Unk'), ('के', 'PREP'), ('एक', 'QFNUM'), ('दिन', 'NN'), ('में', 'PREP'), (',', 'PUNC'), ('जंगल', 'Unk'), ('के', 'PREP'), ('एक', 'QFNUM'), ('शेर', 'Unk'), ('को', 'PREP'), ('बहुत', 'INTF'), ('जोरों', 'NN'), ('से', 'PREP'), ('भूख', 'Unk'), ('लगी', 'VFM'), ('.', 'SYM'), ('इसलिए', 'RB'), ('वो', 'Unk'), ('इधर', 'Unk'), ('उधर', 'RB'), ('खाने', 'Unk'), ('की', 'PREP'), ('तलाश', 'NN'), ('करने', 'VNN'), ('लगा', 'VFM'), ('.', 'SYM'), ('कुछ', 'QF'), ('देर', 'Unk'), ('खोजने', 'Unk'), ('के', 'PREP'), ('बाद', 'PREP'), ('उसे', 'PRP'), ('एक', 'QFNUM'), ('खरगोश', 'Unk'), ('मिला', 'VFM'), (',', 'PUNC'), ('लेकिन', 'CC'), ('उसे', 'PRP'), ('खाने', 'Unk'), ('के', 'PREP'), ('बदले', 'NN'), ('में', 'PREP'), ('उसे', 'PRP'), ('उसने', 'PRP'), ('छोड़', 'Unk'), ('दिया', 'VAUX'), ('क्यूंकि', 'Unk'), ('उसे', 'PRP'), ('वो', 'Unk'), ('बहुत', 'INTF'), ('ही', 'RP'), ('छोटा', 'Unk'), ('लगा', 'VFM'), ('.', 'SYM')]
